In [10]:
# GetOldTweet3 사용 준비
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [11]:
# BeautifulSoup4 사용 준비
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup

In [12]:
# 가져올 범위를 정의
# 예제 : 2019-04-21 ~ 2019-04-24

import datetime

days_range = []

start = datetime.datetime.strptime("2019-10-21", "%Y-%m-%d")
end = datetime.datetime.strptime("2019-10-22", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))

# === 설정된 트윗 수집 기간은 2019-04-21 에서 2019-04-24 까지 입니다 ===
# === 총 4일 간의 데이터 수집 ===

=== 설정된 트윗 수집 기간은 2019-10-21 에서 2019-10-21 까지 입니다 ===
=== 총 1일 간의 데이터 수집 ===


In [13]:
# 특정 검색어가 포함된 트윗 검색하기 (quary search)
# 검색어 : 어벤져스, 스포

import time

# 수집 기간 맞추기
start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
            + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('Trump')\
                                           .setSince(start_date)\
                                           .setUntil(end_date)\
                                           .setMaxTweets(-1)

# 수집 with GetOldTweet3
print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
start_time = time.time()

tweet = got.manager.TweetManager.getTweets(tweetCriteria)

print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
print("=== Total num of tweets is {} ===".format(len(tweet)))

# Collecting data start.. from 2019-04-21 to 2019-04-24
# Collecting data end.. 41.39 Minutes
# === Total num of tweets is 22964 ===

=== Total num of tweets is 625076 ===


In [14]:
type(tweet[0])

GetOldTweets3.models.Tweet.Tweet

In [ ]:
# 원하는 변수 골라서 저장하기

from random import uniform
from tqdm import tqdm_notebook

import requests
from bs4 import BeautifulSoup

def get_bs_obj(url):
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    return bs_obj

# initialize
tweet_list = []

for index in tqdm_notebook(tweet):
    
    # 메타데이터 목록 
    username = index.username
    link = index.permalink 
    content = index.text
    tweet_date = index.date.strftime("%Y-%m-%d")
    tweet_time = index.date.strftime("%H:%M:%S")
    retweets = index.retweets
    favorites = index.favorites
    
    # === 유저 정보 수집 시작 ===
    try:
        personal_link = 'https://twitter.com/' + username
        bs_obj = get_bs_obj(personal_link)
        uls = bs_obj.find("ul", {"class": "ProfileNav-list"}).find_all("li")
        div = bs_obj.find("div", {"class": "ProfileHeaderCard-joinDate"}).find_all("span")[1]["title"]


        # 가입일, 전체 트윗 수, 팔로잉 수, 팔로워 수
        #joined_date = div.split('-')[1].strip()
        #num_tweets = uls[0].find("span", {"class": "ProfileNav-value"}).text.strip()
        #num_following = uls[1].find("span", {"class": "ProfileNav-value"}).text.strip()
        #num_follower = uls[2].find("span", {"class": "ProfileNav-value"}).text.strip()
    
    except AttributeError:
        print("=== Attribute error occurs at {} ===".format(link))
        print("link : {}".format(personal_link))   
        pass
     
    # 결과 합치기
    info_list = [tweet_date, tweet_time, username, content, link, retweets, favorites, 
                   ]
    #joined_date,num_tweets,num_following, num_follower
    tweet_list.append(info_list)
    
    # 휴식 
    time.sleep(uniform(1,2))

=== Attribute error occurs at https://twitter.com/AmericaFightHim/status/1186431909569728513 ===
link : https://twitter.com/AmericaFightHim
=== Attribute error occurs at https://twitter.com/PoGirlShines/status/1186431876178829312 ===
link : https://twitter.com/PoGirlShines
=== Attribute error occurs at https://twitter.com/AmericaFightHim/status/1186431355011424256 ===
link : https://twitter.com/AmericaFightHim


In [ ]:
# 파일 저장하기


import pandas as pd

twitter_df = pd.DataFrame(tweet_list, 
                          columns = ["date", "time", "user_name", "text", "link", "retweet_counts", "favorite_counts",])
                                    #"user_created", "user_tweets", "user_followings", "user_followers"])

# csv 파일 만들기
twitter_df.to_csv("sample_twitter_data_{}_to_{}.csv".format(days_range[0], days_range[-1]), index=False)
print("=== {} tweets are successfully saved ===".format(len(tweet_list)))

# === 22964 tweets are successfully saved ===

In [ ]:
# 파일 확인하기

df_tweet = pd.read_csv('sample_twitter_data_{}_to_{}.csv'.format(days_range[0], days_range[-1]))
df_tweet.head(36) # 위에서 10개만 출력